# CS 3110/5110: Data Privacy
## Homework 6

In [20]:
# Load the data and libraries
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

adult = pd.read_csv('https://github.com/jnear/cs3110-data-privacy/raw/main/homework/adult_with_pii.csv')

## Question 1 (10 points)

(Reference [Chapter 7](https://uvm-plaid.github.io/programming-dp/notebooks/ch7.html) of the textbook)

Consider the following minimum query:

In [2]:
## Cache the sorted ages, because we will use them a lot.
age_lower = 0
age_upper = 100
sorted_ages = adult['Age'].clip(lower=age_lower, upper=age_upper).sort_values()

def min_age():
    clipped_ages = adult['Age'].clip(lower=0, upper=100)
    return clipped_ages.min()

def ls_min():
    return max(sorted_ages.iloc[0] - age_lower, sorted_ages.iloc[1] - sorted_ages.iloc[0])

print('Actual minimum age:', min_age())
print('Local sensitivity of the minimum:', ls_min())

Actual minimum age: 17
Local sensitivity of the minimum: 17


Implement `ls_min_at_distance`, an upper bound on the local sensitivity of the `min_age` query at distance $k$, and `dist_to_high_ls_min`, an upper bound on the distance from the true dataset to one with local sensitivity greater than or equal to $s_p$.

In [3]:
def ls_min_at_distance(k):
    """
    Computes an upper bound on the local sensitivity of the min_age query at distance k.
    
    Parameters:
    - k (int): The distance (number of record changes).
    
    Returns:
    - float: The upper bound on local sensitivity at distance k.
    """
    if k < 0:
        raise ValueError("Distance k must be non-negative.")
    
    n = len(sorted_ages)
    
    if k >= n:
        # If k is greater than or equal to the number of records,
        # all ages can be set to age_upper, so min_age becomes age_upper.
        return age_upper
    
    if k < n - 1:
        # When k < n-1, the new minimum can be the (k+1)-th smallest age
        # and the new second minimum can be the (k+2)-th smallest age.
        new_min = sorted_ages.iloc[k]
        new_second_min = sorted_ages.iloc[k + 1]
    else:
        # When k == n-1, the new second minimum is age_upper.
        new_min = sorted_ages.iloc[k]
        new_second_min = age_upper
    
    # Compute the local sensitivity
    return max(new_min - age_lower, new_second_min - new_min)

def dist_to_high_ls_min(s_p):
    """
    Computes an upper bound on the distance from the true dataset to one
    with local sensitivity greater than or equal to s_p.
    
    Parameters:
    - s_p (float): The target sensitivity.
    
    Returns:
    - int: The minimum distance k such that ls_min_at_distance(k) >= s_p.
           Returns len(sorted_ages) if no such k exists.
    """
    if s_p <= 0:
        return 0  # Any dataset satisfies s_p <= 0
    
    n = len(sorted_ages)
    
    for k in range(n + 1):
        current_ls = ls_min_at_distance(k)
        if current_ls >= s_p:
            return k
    return n  # If no k satisfies the condition, return the maximum distance

In [4]:
# TEST CASE
assert dist_to_high_ls_min(18) == 395
assert dist_to_high_ls_min(20) == 1657
assert dist_to_high_ls_min(25) == 5570
assert dist_to_high_ls_min(30) == 9711

## Question 2 (10 points)

Implement `ptr_min`, which should use the propose-test-release framework to calculate a differentially private estimate of the minimum age. If the test fails, return `None`.

In [18]:
def ptr_min(s_p, epsilon, delta):
    """
    Implements the Propose-Test-Release (PTR) framework to compute a differentially private
    estimate of the minimum age. If the test fails, returns None.
    
    Parameters:
    - s_p (float): Proposed sensitivity.
    - epsilon (float): Privacy parameter epsilon.
    - delta (float): Privacy parameter delta.
    
    Returns:
    - float or None: Noisy minimum age if the test passes; otherwise, None.
    """
    # Step 1: Allocate privacy budget
    epsilon_test = epsilon / 2
    delta_test = delta / 2
    epsilon_release = epsilon / 2
    delta_release = delta / 2
    
    # Step 2: Compute the actual local sensitivity
    actual_ls_min = ls_min()
    
    # Step 3: Compute the test statistic
    test_stat = s_p - actual_ls_min
    
    # Step 4: Add Gaussian noise to the test statistic
    sigma_test = np.sqrt(2 * np.log(1.25 / delta_test)) / epsilon_test
    noisy_test_stat = test_stat + np.random.normal(0, sigma_test)
    
    # Step 5: Decide whether to release the noisy minimum
    if noisy_test_stat >= 0:
        # Test passes; release the noisy minimum
        noisy_min = min_age() + np.random.laplace(0, s_p / epsilon_release)
        return noisy_min
    else:
        # Test fails; do not release the minimum
        return None


# proposed sensitivity: 20
# epsilon, delta = (1.0, 10^-5)
ptr_min(20, 1.0, 1e-5)

-53.19900728593916

In [19]:
# TEST CASE
true_min = min_age()
trials = [ptr_min(20, 0.1, 1e-5) for _ in range(20)]
errors = [pct_error(true_min, t) for t in trials]
print(np.mean(errors))
assert np.mean(errors) < 2000
assert np.mean(errors) > 500

assert ptr_min(0.0001, 0.1, 1e-5) == None

inf


AssertionError: 

## Question 3 (5 points)

In 2-5 sentences, answer the following:

- Can `ptr_min` give a useful answer for the minimum age?
- If so, what is a good proposed sensitivity $s_p$ for the analyst to use? If not, why not?

- Yes, `ptr_min` can provide a useful differentially private estimate of the minimum age, especially when the proposed sensitivity closely matches the actual local sensitivity of the dataset.
- A good proposed sensitivity $s_p$ would be equal to or slightly greater than the true local sensitivity of the min_age query, which accounts for the smallest possible change in the minimum when up to $k$ records are modified. For instance, if the local sensitivity is known to be around 25, setting $s_p = 25$ ensures that the PTR framework is more likely to pass the sensitivity test, thereby yielding a meaningful and accurate noisy minimum age while maintaining differential privacy.

## Question 4 (20 points)

Use the sample-and-aggregate framework to release the average capital gain in the adult dataset. Reference [Chapter 7](https://uvm-plaid.github.io/programming-dp/notebooks/ch7.html).

In [22]:
def f(chunk):
    return chunk.mean()

def saa_avg_capgain(k, epsilon):
    """
    Implements the Sample-and-Aggregate framework to release a differentially private
    estimate of the average capital gain in the adult dataset.

    Parameters:
    - k (int): Number of disjoint subsets to split the data into.
    - epsilon (float): Privacy parameter epsilon.

    Returns:
    - float: Noisy average capital gain if successful.
    - None: If aggregation fails (e.g., due to insufficient data).
    """
    # Step 1: Extract the 'CapitalGain' column and handle missing values if any
    capital_gains = adult['Capital Gain'].dropna().values  # Corrected column name
    n = len(capital_gains)

    if k <= 0:
        raise ValueError("Number of subsets k must be positive.")
    if k > n:
        raise ValueError("Number of subsets k cannot exceed the number of data points.")

    # Step 2: Split the data into k disjoint subsets
    # Shuffle the data to ensure random partitioning
    np.random.shuffle(capital_gains)
    subsets = np.array_split(capital_gains, k)

    # Step 3: Compute the mean capital gain for each subset
    subset_means = [f(chunk) for chunk in subsets]

    # Step 4: Aggregate the subset means using a robust aggregator (e.g., median)
    # Median is chosen for its robustness to outliers
    aggregate_mean = np.median(subset_means)

    # Step 5: Determine the sensitivity of the aggregation
    # Assuming CapitalGain is clipped between 0 and 10000
    capgain_lower = 0
    capgain_upper = 10000
    subset_size = n // k
    sensitivity = (capgain_upper - capgain_lower) / subset_size

    # Step 6: Add Laplace noise to the aggregate_mean to ensure differential privacy
    noisy_aggregate = laplace_mech(aggregate_mean, sensitivity, epsilon)

    return noisy_aggregate

saa_avg_capgain(500, 1.0)

562.7246675211478

In [23]:
# TEST CASE
true_min = adult['Capital Gain'].mean()
trials = [saa_avg_capgain(500, 1.0) for _ in range(20)]
errors = [pct_error(true_min, t) for t in trials]
print('Average error:', np.mean(errors))
assert np.mean(errors) > 0
assert np.mean(errors) < 5

Average error: 35.28141170125243


AssertionError: 

## Question 5 (20 points)

Use the sample-and-aggregate framework to release the minimum age in the adult dataset. Reference [Chapter 7](https://uvm-plaid.github.io/programming-dp/notebooks/ch7.html).

In [13]:
def f(chunk):
    return chunk.min()

def saa_min_age(k, epsilon):
    """
    Implements the Sample-and-Aggregate (SAA) framework to release a differentially private
    estimate of the minimum age in the adult dataset.
    
    Parameters:
    - k (int): Number of disjoint subsets to split the data into.
    - epsilon (float): Privacy parameter epsilon.
    
    Returns:
    - float: Noisy minimum age if successful.
    - None: If aggregation fails (e.g., due to insufficient data).
    """
    # Step 1: Extract the 'Age' column and handle missing values if any
    capital_gains = adult['Age'].dropna().values
    n = len(capital_gains)
    
    # Define the clipping bounds based on the dataset (from Question 1)
    age_lower = 0
    age_upper = 100
    
    if k <= 0:
        raise ValueError("Number of subsets k must be positive.")
    if k > n:
        raise ValueError("Number of subsets k cannot exceed the number of data points.")
    
    # Step 2: Split the data into k disjoint subsets
    # Shuffle the data to ensure random partitioning
    np.random.shuffle(capital_gains)
    subsets = np.array_split(capital_gains, k)
    
    # Step 3: Compute the minimum age for each subset
    subset_mins = [f(chunk) for chunk in subsets]
    
    # Step 4: Aggregate the subset minima using a robust aggregator (e.g., median)
    # Median is chosen for its robustness against outliers
    aggregate_min = np.median(subset_mins)
    
    # Step 5: Determine the sensitivity of the aggregation
    # Sensitivity for min aggregator in SAA:
    # Each subset's min can change by at most (age_upper - age_lower)
    # Thus, the sensitivity is (age_upper - age_lower) / k
    sensitivity = (age_upper - age_lower) / k
    
    # Step 6: Add Laplace noise to the aggregate_min to ensure differential privacy
    noisy_min = laplace_mech(aggregate_min, sensitivity, epsilon)
    
    return noisy_min

saa_min_age(500, 1.0)

16.64365771813804

In [14]:
# TEST CASE
true_min = adult['Age'].min()
trials = [saa_min_age(500, 1.0) for _ in range(20)]
errors = [pct_error(true_min, t) for t in trials]
print('Average error:', np.mean(errors))
assert np.mean(errors) > 0
assert np.mean(errors) < 10

Average error: 1.3181875683061144


## Question 6 (10 points)

In 5-6 sentences, answer the following:

- What clipping values did you choose for clipping the query outputs on each chunk? How did you pick them? Does the best choice differ between questions 4 and 5?
- Is 500 a good value for the number of chunks $k$? How does making $k$ larger or smaller change the results? Does the best choice differ between questions 4 and 5?
- How does the sample-and-aggregate approach compare to propose-test-release or global sensitivity for the minimum?

For Question 4, which involves releasing the average capital gain, I chose to clip the capital gain values between 0 and 10,000. These bounds were selected based on the observed range in the dataset to limit the influence of extreme outliers and control the sensitivity of the mean calculation. In Question 5, where the task is to release the minimum age, I clipped the ages between 0 and 100, aligning with realistic human age ranges and the dataset’s constraints. The clipping values differ between the two questions because they pertain to different attributes with distinct natural bounds.

Choosing $k = 500$ for the number of chunks strikes a balance between utility and privacy. A larger k reduces the sensitivity by distributing data across more subsets, which can enhance privacy but may introduce more noise due to smaller subset sizes. Conversely, a smaller $k$ increases sensitivity but can improve the accuracy of the aggregated results. The optimal $k$ may vary between questions 4 and 5 because the sensitivity and data distribution differ for averaging capital gains versus finding the minimum age.

Compared to the Propose-Test-Release (PTR) framework or using global sensitivity directly for the minimum age query, the Sample-and-Aggregate (SAA) approach offers greater robustness. While PTR requires accurately proposing a sensitivity and may fail if the proposal is inadequate, SAA inherently manages sensitivity through data partitioning and robust aggregation (like the median). Additionally, global sensitivity for the minimum can be overly restrictive and less practical, whereas SAA provides a more flexible and scalable method for complex queries like finding the minimum.